# Semi-structured eval: baseline retriever

We will test retrieval of table information from the `Semi-structured Reports` dataset using various methods.

## Pre-requisites

In [1]:
%pip install --quiet -U langchain langsmith langchain_benchmarks
%pip install --quiet chromadb openai pypdf pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Dataset

In [2]:
import os
from pathlib import Path

from langchain_benchmarks import clone_public_dataset, registry
from langchain_benchmarks.rag.tasks.semi_structured_reports import get_file_names

# Task
task = registry["Semi-structured Reports"]

# Files used
paths = list(get_file_names())
files = [str(p) for p in paths]

### TODO: Replace when dataset is updated
#dir = "/Users/rlm/Desktop/Eval_Sets/semi_structured_reports/"
dir = "/mnt/c/Users/taqi_/OneDrive/Desktop/semi_structured_reports/"
files = [f for f in os.listdir(dir) if f.endswith(".pdf")]

## Load

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


def load_and_split(file):
    """
    Load and split PDF files
    """

    loader = PyPDFLoader(file)
    pdf_pages = loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=100, chunk_overlap=50
    )

    # Get chunks
    docs = text_splitter.split_documents(pdf_pages)
    texts = [d.page_content for d in docs]
    print(f"There are {len(texts)} text elements")
    return texts


texts = []
for fi in files:
    texts.extend(load_and_split(dir + fi))

There are 285 text elements
There are 67 text elements
There are 146 text elements
There are 130 text elements
There are 77 text elements
There are 59 text elements


## Index

In [4]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore_baseline = Chroma.from_texts(
    texts=texts, collection_name="baseline-100-token", embedding=OpenAIEmbeddings()
)

retriever_baseline = vectorstore_baseline.as_retriever()

## RAG

In [5]:
from typing import List

from langchain.schema.document import Document
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough


def format_context(docs: List[Document]) -> str:
    return "\n\n".join([i.page_content for i in docs])


def rag_chain(retriever):
    """
    RAG chain
    """

    # Prompt template
    template = """Answer the question based only on the following context, which can include text and tables:
    {context}
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    # LLM
    model = ChatOpenAI(temperature=0, model="gpt-4")

    # RAG pipeline
    chain = (
        {
            "context": retriever | RunnableLambda(format_context),
            "question": RunnablePassthrough(),
        }
        | prompt
        | model
        | StrOutputParser()
    )
    return chain


# Create RAG chain
chain = rag_chain(retriever_baseline)

## Eval

In [6]:
### TODO: Replace with public dataset

import uuid

import pandas as pd
from langsmith import Client

# Read
df = pd.read_csv(dir + "semi_structured_reports.csv")

# Dataset
client = Client()
dataset_name = f"Semi-Structured-Eval"
existing_datasets = list(client.list_datasets(dataset_name=dataset_name))
if existing_datasets:
    # read existing dataset
    dataset = client.read_dataset(dataset_name=dataset_name)
else:
    dataset = client.create_dataset(dataset_name=dataset_name)
    # Populate dataset
    for _, row in df.iterrows():
        # Get Q, A
        q = row["Question"]
        a = row["Answer"]

        # Use the values in your function
        client.create_example(
            inputs={"question": q}, outputs={"answer": a}, dataset_id=dataset.id
        )

In [7]:
from functools import partial

from langsmith.client import Client
from langchain.smith import RunEvalConfig
from langchain_benchmarks.rag import get_eval_config

eval_config = RunEvalConfig(
    evaluators=["cot_qa"],
)

def run_eval(chain, eval_run_name):
    """
    Run eval
    """
    client = Client()
    test_run = client.run_on_dataset(
        ### TODO: Replace with public dataset
        dataset_name="Semi-Structured-Eval",
        llm_or_chain_factory=lambda: (lambda x: x["question"]) | chain,
        evaluation=eval_config,
        verbose=True,
        project_name=eval_run_name,
    )


# Experiments
chain_map = {
    "baseline-100tok": chain,
}

run_id = str(uuid.uuid4())
for project_name, chain in chain_map.items():
    run_eval(chain, project_name + "_" + run_id)

View the evaluation results for project 'baseline-100tok_092dede3-b410-480d-93fa-d5c7d8ccc019' at:
https://smith.langchain.com/o/530c4d06-5640-4c0f-94fe-0be7b769531f/datasets/7f6e64ae-218b-47cc-83a9-1991bd5fbfc8/compare?selectedSessions=620abf91-9c49-4b00-a029-e5481a167173

View all tests for Dataset Semi-Structured-Eval at:
https://smith.langchain.com/o/530c4d06-5640-4c0f-94fe-0be7b769531f/datasets/7f6e64ae-218b-47cc-83a9-1991bd5fbfc8
[------------------------------------------------->] 20/20

,output,feedback.COT Contextual Accuracy,error,execution_time
count,20,20.000000,0,20.000000
unique,20,NaN,0,NaN
top,The text does not provide information on the p...,NaN,NaN,NaN
freq,1,NaN,NaN,NaN
mean,NaN,0.400000,NaN,6.118559
std,NaN,0.502625,NaN,1.408516
min,NaN,0.000000,NaN,3.398590
25%,NaN,0.000000,NaN,5.042491
50%,NaN,0.000000,NaN,5.771672
75%,NaN,1.000000,NaN,7.160996
